In [1]:
%matplotlib inline
import numpy as np
import matplotlib as plt
import pandas as pd
import pymrio
import os
from requests import RequestException
import csv
import re
import json
import copy
import ast
import operator
import math
import time
from collections import Counter

# Data 

A.txt files contain MRIOT, representing the financial expenditures (in Euros) from each industry for each industry. Provides Euros of output from row industries to produce an Euro of output in a column industry

F.txt contain the CO2 - combustion - air values for each CountryxIndustry. We only care about the 3 carbon-intensive industries for this: coal mining, petroleum and natural gas. This allows for calculation using the Tier 1 method of carbon direct emissions.

## This Notebook largely uses pymrio. documentation can be found at [this link](https://pymrio.readthedocs.io/en/latest/)

Due to complexity, this file will be run separately for each year to obtain EXIOBASE data. Every time this file runs, change year parameters and data files to the appropriate year. In OrganizeTrade, the data across all years will be concatenated.

In [2]:
exio3 = pymrio.parse_exiobase3(path='../Data/2_Carbon/exio3/ixi_zip/IOT_2010_ixi.zip')

### Below is for a preview of data

In [3]:
# exio3.calc_all()

In [4]:
# emissions_df = exio3.satellite.D_cba
# emissions_df

In [5]:
# co2_df  = emissions_df[emissions_df.index.isin(['CO2 - combustion - air'])]

# constant = 1000000
# scaled_co2_df = co2_df[co2_df.columns].apply(lambda x: x/constant)
# scaled_co2_df

# scaled_co2_df gives us CO2 per euros in df

In [6]:
# scaled_co2_df.to_csv('../Data/2_Carbon/co2_rates_for_testing.csv')

# Aggregation: Finding Carbon Emissions for Each Industry X Country in Current Year

Satellite includes a row for CO2 combustion that represents total direct emissions of a product in a country in this current year.

In [1]:
# Check to see available extension tables with parsed exio3

list(exio3.get_extensions())

NameError: name 'exio3' is not defined

In [8]:
exio3.satellite.F

region                                                                    AT  \
sector                                             Cultivation of paddy rice   
Taxes less subsidies on products purchased: Total                          0   
Other net taxes on production                                              0   
Compensation of employees; wages, salaries, & e...                         0   
Compensation of employees; wages, salaries, & e...                         0   
Compensation of employees; wages, salaries, & e...                         0   
...                                                                      ...   
Energy Carrier Net TMAR                                                    0   
Energy Carrier Net TOTH                                                    0   
Energy Carrier Net TRAI                                                    0   
Energy Carrier Net TROA                                                    0   
Energy Carrier Net LOSS                                                    0   

region                                                                   \
sector                                             Cultivation of wheat   
Taxes less subsidies on products purchased: Total              5.405968   
Other net taxes on production                                -11.312479   
Compensation of employees; wages, salaries, & e...             1.029400   
Compensation of employees; wages, salaries, & e...            14.193025   
Compensation of employees; wages, salaries, & e...             2.256036   
...                                                                 ...   
Energy Carrier Net TMAR                                        0.000000   
Energy Carrier Net TOTH                                        0.000000   
Energy Carrier Net TRAI                                        0.000000   
Energy Carrier Net TROA                                     1234.000000   
Energy Carrier Net LOSS                                        0.000000   

region                                                                               \
sector                                             Cultivation of cereal grains nec   
Taxes less subsidies on products purchased: Total                          9.526391   
Other net taxes on production                                            -21.653771   
Compensation of employees; wages, salaries, & e...                         1.866267   
Compensation of employees; wages, salaries, & e...                        25.731467   
Compensation of employees; wages, salaries, & e...                         4.090116   
...                                                                             ...   
Energy Carrier Net TMAR                                                    0.000000   
Energy Carrier Net TOTH                                                    0.000000   
Energy Carrier Net TRAI                                                    0.000000   
Energy Carrier Net TROA                                                  200.000000   
Energy Carrier Net LOSS                                                    0.000000   

region                                                                                     \
sector                                             Cultivation of vegetables, fruit, nuts   
Taxes less subsidies on products purchased: Total                               11.205557   
Other net taxes on production                                                  -45.004973   
Compensation of employees; wages, salaries, & e...                               4.001617   
Compensation of employees; wages, salaries, & e...                              55.172962   
Compensation of employees; wages, salaries, & e...                               8.769955   
...                                                                                   ...   
Energy Carrier Net TMAR                                                          0.000000   
Energy Carrier

#### diag_stressor takes one row of the F matrix and diagonalize to the full region/sector format. Footprints calculation based on this matrix show the flow of embodied stressors from the source region/sector (row index) to the final consumer (column index).

In [9]:
co2_diag = exio3.satellite.diag_stressor(('CO2 - combustion - air'), name = 'co2_diag')
co2_diag.F.head(10)

region                                                               AT  \
sector                                        Cultivation of paddy rice   
region sector                                                             
AT     Cultivation of paddy rice                                    0.0   
       Cultivation of wheat                                         0.0   
       Cultivation of cereal grains nec                             0.0   
       Cultivation of vegetables, fruit, nuts                       0.0   
       Cultivation of oil seeds                                     0.0   
       Cultivation of sugar cane, sugar beet                        0.0   
       Cultivation of plant-based fibers                            0.0   
       Cultivation of crops nec                                     0.0   
       Cattle farming                                               0.0   
       Pigs farming                                                 0.0   

region                                                              \
sector                                        Cultivation of wheat   
region sector                                                        
AT     Cultivation of paddy rice                      0.000000e+00   
       Cultivation of wheat                           1.722999e+08   
       Cultivation of cereal grains nec               0.000000e+00   
       Cultivation of vegetables, fruit, nuts         0.000000e+00   
       Cultivation of oil seeds                       0.000000e+00   
       Cultivation of sugar cane, sugar beet          0.000000e+00   
       Cultivation of plant-based fibers              0.000000e+00   
       Cultivation of crops nec                       0.000000e+00   
       Cattle farming                                 0.000000e+00   
       Pigs farming                                   0.000000e+00   

region                                                                          \
sector                                        Cultivation of cereal grains nec   
region sector                                                                    
AT     Cultivation of paddy rice                                  0.000000e+00   
       Cultivation of wheat                                       0.000000e+00   
       Cultivation of cereal grains nec                           1.894334e+08   
       Cultivation of vegetables, fruit, nuts                     0.000000e+00   
       Cultivation of oil seeds                                   0.000000e+00   
       Cultivation of sugar cane, sugar beet                      0.000000e+00   
       Cultivation of plant-based fibers                          0.000000e+00   
       Cultivation of crops nec                                   0.000000e+00   
       Cattle farming                                             0.000000e+00   
       Pigs farming                                               0.000000e+00   

region                                                                                \
sector                                        Cultivation of vegetables, fruit, nuts   
region sector                                                                          
AT     Cultivation of paddy rice                                        0.000000e+00   
       Cultivation of wheat                                             0.000000e+00   
       Cultivation of cereal grains nec                                 0.000000e+00   
       Cultivation of vegetables, fruit, nuts                           1.123415e+08   
       Cultivation of oil seeds                                         0.000000e+00   
       Cultivation of sugar cane, sugar beet                            0.000000e+00   
       Cultivation of plant-based fibers                                0.000000e+00   
       Cultivation of crops nec                                         0.000000e+00   
       Cattle farming                                                   0.0000

#### calc_all() calculates missing parts of the IOSystem and all extensions.

In [10]:
exio3.co2_diag = co2_diag
exio3.calc_all()

In [11]:
exio3.co2_diag.D_cba.head()

region                                                               AT  \
sector                                        Cultivation of paddy rice   
region sector                                                             
AT     Cultivation of paddy rice                               0.000000   
       Cultivation of wheat                                    1.173052   
       Cultivation of cereal grains nec                        1.370299   
       Cultivation of vegetables, fruit, nuts                  0.091968   
       Cultivation of oil seeds                                0.196996   

region                                                              \
sector                                        Cultivation of wheat   
region sector                                                        
AT     Cultivation of paddy rice                      0.000000e+00   
       Cultivation of wheat                           2.035509e+07   
       Cultivation of cereal grains nec               1.657113e+02   
       Cultivation of vegetables, fruit, nuts         3.681902e+01   
       Cultivation of oil seeds                       1.898058e+01   

region                                                                          \
sector                                        Cultivation of cereal grains nec   
region sector                                                                    
AT     Cultivation of paddy rice                                  0.000000e+00   
       Cultivation of wheat                                       3.699475e+02   
       Cultivation of cereal grains nec                           3.388182e+07   
       Cultivation of vegetables, fruit, nuts                     1.013928e+02   
       Cultivation of oil seeds                                   4.428468e+01   

region                                                                                \
sector                                        Cultivation of vegetables, fruit, nuts   
region sector                                                                          
AT     Cultivation of paddy rice                                        0.000000e+00   
       Cultivation of wheat                                             2.221641e+03   
       Cultivation of cereal grains nec                                 2.951648e+03   
       Cultivation of vegetables, fruit, nuts                           6.103087e+07   
       Cultivation of oil seeds                                         2.355713e+02   

region                                                                  \
sector                                        Cultivation of oil seeds   
region sector                                                            
AT     Cultivation of paddy rice                          0.000000e+00   
       Cultivation of wheat                               1.183119e+02   
       Cultivation of cereal grains nec                   1.403405e+02   
       Cultivation of vegetables, fruit, nuts             4.406956e+01   
       Cultivation of oil seeds                           5.750042e+06   

region                                                                               \
sector                                        Cultivation of sugar cane, sugar beet   
region sector                                                                         
AT     Cultivation of paddy rice                                           0.000000   
       Cultivation of wheat                                                8.343994   
       Cultivation of cereal grains nec                                   11.162788   
       Cultivation of vegetables, fruit, nuts                              2.380651   
       Cultivation of oil seeds                                            0.753903   

region                                                                           \
sector                                        Cultivation of plant-based fibers   
region sector       

In [12]:
df_with_co2 = exio3.co2_diag.D_cba.groupby(level='sector', axis=0).sum()
df_with_co2.head(40)

region                                                                    AT  \
sector                                             Cultivation of paddy rice   
sector                                                                         
Activities auxiliary to financial intermediatio...              4.930548e+02   
Activities of membership organisation n.e.c. (91)               7.910753e+00   
Air transport (62)                                              1.423628e+04   
Aluminium production                                            6.697324e+02   
Animal products nec                                             1.358293e+02   
Biogasification of food waste, incl. land appli...              2.906147e-01   
Biogasification of paper, incl. land application                2.218252e+00   
Biogasification of sewage slugde, incl. land ap...              4.312359e+00   
Casting of metals                                               9.390085e+02   
Cattle farming                                                  3.320070e+02   
Chemicals nec                                                   4.016690e+04   
Collection, purification and distribution of wa...              1.322984e+02   
Composting of food waste, incl. land application                5.102537e+00   
Composting of paper and wood, incl. land applic...              6.647735e-01   
Computer and related activities (72)                            3.963173e+01   
Construction (45)                                               1.063341e+03   
Copper production                                               3.739143e+02   
Cultivation of cereal grains nec                                7.569953e+02   
Cultivation of crops nec                                        4.702917e+02   
Cultivation of oil seeds                                        4.445558e+02   
Cultivation of paddy rice                                       7.024966e+06   
Cultivation of plant-based fibers                               1.610295e+03   
Cultivation of sugar cane, sugar beet                           2.236531e+02   
Cultivation of vegetables, fruit, nuts                          5.389267e+02   
Cultivation of wheat                                            5.994254e+03   
Distribution and trade of electricity                           2.417529e+04   
Education (80)                                                 -6.511855e-01   
Extra-territorial organizations and bodies                      3.013060e-09   
Extraction of crude petroleum and services rela...              1.088432e+04   
Extraction of natural gas and services related ...              1.483704e+04   
Extraction, liquefaction, and regasification of...              2.130605e+03   
Financial intermediation, except insurance and ...              1.115048e+03   
Fishing, operating of fish hatcheries and fish ...              1.211794e+02   
Forestry, logging and related service activitie...              1.670950e+02   
Health and social work (85)                                     8.022157e+00   
Hotels and restaurants (55)                                     9.896502e+02   
Incineration of waste: Food                                     2.869062e+00   
Incineration of waste: Metals and Inert materials               7.953287e+01   
Incineration of waste: Oil/Hazardous waste                      3.619526e+01   
Incineration of waste: Paper                                    5.553553e+00   

region                                                                   \
sector                                             Cultivation of wheat   
sector                                                                    
Activities auxiliary to financial intermediatio...         1.097508e+04   
Activities of membership organisation n.e.c. (91)          6.640049e+01   
Air transport (62)                                         2.309676e+05   
Aluminium production                                       1.355436e+04   
Animal products nec               

#### I want to focus on the three industries that have the highest carbon emissions contributions, according to the Method 1. However, this can be changed later on in the process of my research if I want to use ALL carbon emissions contributions.

In [13]:
co2_final_df_three_industries  = df_with_co2[df_with_co2.index.isin(['Mining of coal and lignite; extraction of peat (10)',
                                                   'Extraction of crude petroleum and services related to crude oil extraction, excluding surveying',
                                                   'Extraction of natural gas and services related to natural gas extraction, excluding surveying'])]
co2_final_df_three_industries

region                                                                    AT  \
sector                                             Cultivation of paddy rice   
sector                                                                         
Extraction of crude petroleum and services rela...              10884.322203   
Extraction of natural gas and services related ...              14837.038300   
Mining of coal and lignite; extraction of peat ...              46089.439950   

region                                                                   \
sector                                             Cultivation of wheat   
sector                                                                    
Extraction of crude petroleum and services rela...         4.900579e+05   
Extraction of natural gas and services related ...         1.521866e+06   
Mining of coal and lignite; extraction of peat ...         3.476144e+05   

region                                                                               \
sector                                             Cultivation of cereal grains nec   
sector                                                                                
Extraction of crude petroleum and services rela...                     1.191910e+06   
Extraction of natural gas and services related ...                     2.444021e+06   
Mining of coal and lignite; extraction of peat ...                     5.805078e+05   

region                                                                                     \
sector                                             Cultivation of vegetables, fruit, nuts   
sector                                                                                      
Extraction of crude petroleum and services rela...                           3.677716e+06   
Extraction of natural gas and services related ...                           9.108738e+06   
Mining of coal and lignite; extraction of peat ...                           3.370443e+06   

region                                                                       \
sector                                             Cultivation of oil seeds   
sector                                                                        
Extraction of crude petroleum and services rela...             1.471618e+06   
Extraction of natural gas and services related ...             6.567360e+06   
Mining of coal and lignite; extraction of peat ...             5.495124e+05   

region                                                                                    \
sector                                             Cultivation of sugar cane, sugar beet   
sector                                                                                     
Extraction of crude petroleum and services rela...                          10161.342848   
Extraction of natural gas and services related ...                         -17361.353202   
Mining of coal and lignite; extraction of peat ...                           3007.246633   

region                                                                                \
sector                                             Cultivation of plant-based fibers   
sector                                                                                 
Extraction of crude petroleum and services rela...                      10739.907917   
Extraction of natural gas and services related ...                      23795.531444   
Mining of coal and lignite; extraction of peat ...                     -54006.513725   

region                                                                       \
sector                                             Cultivation of crops nec   
sector                                                                        
Extraction of crude petroleum and services rela...            400591.472696   
Extraction of natural gas and services related ...            401641.246179   
Mining of coal and lignite; extrac

In [14]:
co2_final = co2_final_df_three_industries.append(co2_final_df_three_industries.sum().rename('Total CO2 Emissions from Intensive Industries'))

In [15]:
co2_final.index.values

array(['Extraction of crude petroleum and services related to crude oil extraction, excluding surveying',
       'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Mining of coal and lignite; extraction of peat (10)',
       'Total CO2 Emissions from Intensive Industries'], dtype=object)

#### Finally, I have the df that shows the total CO2 emissions from intensive industries for each Industry X Country.

In [16]:
co2_final_total = co2_final.drop(['Extraction of crude petroleum and services related to crude oil extraction, excluding surveying',
                                  'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Mining of coal and lignite; extraction of peat (10)'])
co2_final_total

region                                                               AT  \
sector                                        Cultivation of paddy rice   
sector                                                                    
Total CO2 Emissions from Intensive Industries              71810.800453   

region                                                              \
sector                                        Cultivation of wheat   
sector                                                               
Total CO2 Emissions from Intensive Industries         2.359538e+06   

region                                                                          \
sector                                        Cultivation of cereal grains nec   
sector                                                                           
Total CO2 Emissions from Intensive Industries                     4.216438e+06   

region                                                                                \
sector                                        Cultivation of vegetables, fruit, nuts   
sector                                                                                 
Total CO2 Emissions from Intensive Industries                           1.615690e+07   

region                                                                  \
sector                                        Cultivation of oil seeds   
sector                                                                   
Total CO2 Emissions from Intensive Industries             8.588490e+06   

region                                                                               \
sector                                        Cultivation of sugar cane, sugar beet   
sector                                                                                
Total CO2 Emissions from Intensive Industries                          -4192.763722   

region                                                                           \
sector                                        Cultivation of plant-based fibers   
sector                                                                            
Total CO2 Emissions from Intensive Industries                     -19471.074364   

region                                                                  \
sector                                        Cultivation of crops nec   
sector                                                                   
Total CO2 Emissions from Intensive Industries             1.160050e+06   

region                                                                      \
sector                                        Cattle farming  Pigs farming   
sector                                                                       
Total CO2 Emissions from Intensive Industries   2.437807e+06  1.011877e+06   

region                                         ...                       WM  \
sector                                         ... Landfill of waste: Paper   
sector                                         ...                            
Total CO2 Emissions from Intensive Industries  ...             1.220088e+07   

region                                                                    \
sector                                        Landfill of waste: Plastic   
sector                                                                     
Total CO2 Emissions from Intensive Industries               6.740236e+06   

region                                                                                  \
sector                                        Landfill of waste: Inert/metal/hazardous   
sector                                                                                   
Total CO2 Emissions from Intensive Industries                             1.268608e+07   

region                                                                     \
sector                                        Landfill of waste: Textiles   
sector    

# Mapping Industry to Product

Because the final dataset is at the product level, the industries must be matched to their corresponding product using the concordance table provided by EXIOBASE 3 [here](https://zenodo.org/record/4277368#.YCn_P89Khqs)

In [17]:
co2_final_total.columns

MultiIndex([('AT',                           'Cultivation of paddy rice'),
            ('AT',                                'Cultivation of wheat'),
            ('AT',                    'Cultivation of cereal grains nec'),
            ('AT',              'Cultivation of vegetables, fruit, nuts'),
            ('AT',                            'Cultivation of oil seeds'),
            ('AT',               'Cultivation of sugar cane, sugar beet'),
            ('AT',                   'Cultivation of plant-based fibers'),
            ('AT',                            'Cultivation of crops nec'),
            ('AT',                                      'Cattle farming'),
            ('AT',                                        'Pigs farming'),
            ...
            ('WM',                            'Landfill of waste: Paper'),
            ('WM',                          'Landfill of waste: Plastic'),
            ('WM',            'Landfill of waste: Inert/metal/hazardous'),
         

In [18]:
ip_concordance = pd.read_csv('../Data/2_Carbon/EXIOBASE20p_EXIOBASE20i.txt', delimiter = '\t', index_col=False)

ip_concordance

,Unnamed: 0,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
0,Paddy rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wheat,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cereal grains nec,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Vegetables, fruit, nuts",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Oil seeds,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Membership organisation services n.e.c. (91),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
196,"Recreational, cultural and sporting services (92)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
197,Other services (93),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
198,Private households with employed persons (95),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
# create dictionary with industry as key, product(s) as value(s)

ip_dict = {}
keys = []
values = []

industries = list(ip_concordance.columns)
industries.pop(0)

products = list(ip_concordance['Unnamed: 0'])

for column in ip_concordance:
    keys.append(column)
    row_vals = ip_concordance.loc[ip_concordance[column] == 1]
    for index, row in row_vals.iterrows():
        titles = ip_concordance.iloc[index,0]
        values.append(titles)

keys.pop(0)

ip_dict = dict(zip(keys, values))

print(ip_dict)

{'Cultivation of paddy rice': 'Paddy rice', 'Cultivation of wheat': 'Wheat', 'Cultivation of cereal grains nec': 'Cereal grains nec', 'Cultivation of vegetables, fruit, nuts': 'Vegetables, fruit, nuts', 'Cultivation of oil seeds': 'Oil seeds', 'Cultivation of sugar cane, sugar beet': 'Sugar cane, sugar beet', 'Cultivation of plant-based fibers': 'Plant-based fibers', 'Cultivation of crops nec': 'Crops nec', 'Cattle farming': 'Cattle', 'Pigs farming': 'Pigs', 'Poultry farming': 'Poultry', 'Meat animals nec': 'Meat animals nec', 'Animal products nec': 'Animal products nec', 'Raw milk': 'Raw milk', 'Wool, silk-worm cocoons': 'Wool, silk-worm cocoons', 'Manure treatment (conventional) and land application': 'Manure (conventional treatment)', 'Manure treatment (biogas) and land application': 'Manure (biogas treatment)', 'Forestry, logging and related service activities (02)': 'Products of forestry, logging and related services (02)', 'Fishing, operating of fish hatcheries and fish farms; se

In [20]:
# map industry names to product names

co2_final_total_products = co2_final_total.rename(columns=ip_dict)

In [21]:
co2_final_total_products_flipped = co2_final_total_products.melt()
co2_final_total_products_flipped


,region,sector,value
0,AT,Paddy rice,7.181080e+04
1,AT,Wheat,2.359538e+06
2,AT,Cereal grains nec,4.216438e+06
3,AT,"Vegetables, fruit, nuts",1.615690e+07
4,AT,Oil seeds,8.588490e+06
...,...,...,...
7982,WM,Transportation services via pipelines,2.706219e+08
7983,WM,Sea and coastal water transportation services,3.811673e+08
7984,WM,Inland water transportation services,2.510483e+08
7985,WM,Air transport services (62),4.015510e+07


# Map product exio codes

exio codes are unique to EXIOBASE and can later be mapped to HS6 using a concordance table provided in same link above

In [22]:
df_codes = pd.read_excel('../Data/2_Carbon/Exiobase_supp.xlsx', 'Products')
df_codes

,Unnamed: 0,Nbr,Name,CodeNr,CodeTxt
0,NaN,1,Paddy rice,p01.a,C_PARI
1,NaN,2,Wheat,p01.b,C_WHEA
2,NaN,3,Cereal grains nec,p01.c,C_OCER
3,NaN,4,"Vegetables, fruit, nuts",p01.d,C_FVEG
4,NaN,5,Oil seeds,p01.e,C_OILS
...,...,...,...,...,...
195,NaN,196,Membership organisation services n.e.c. (91),p91,C_ORGA
196,NaN,197,"Recreational, cultural and sporting services (92)",p92,C_RECR
197,NaN,198,Other services (93),p93,C_OSER
198,NaN,199,Private households with employed persons (95),p95,C_PRHH


In [23]:
codes_dict = pd.Series(df_codes.CodeNr.values,index=df_codes.Name).to_dict()
codes_dict

{'Paddy rice': 'p01.a',
 'Wheat': 'p01.b',
 'Cereal grains nec': 'p01.c',
 'Vegetables, fruit, nuts': 'p01.d',
 'Oil seeds': 'p01.e',
 'Sugar cane, sugar beet': 'p01.f',
 'Plant-based fibers': 'p01.g',
 'Crops nec': 'p01.h',
 'Cattle': 'p01.i',
 'Pigs': 'p01.j',
 'Poultry': 'p01.k',
 'Meat animals nec': 'p01.l',
 'Animal products nec': 'p01.m',
 'Raw milk': 'p01.n',
 'Wool, silk-worm cocoons': 'p01.o',
 'Manure (conventional treatment)': 'p01.w.1',
 'Manure (biogas treatment)': 'p01.w.2',
 'Products of forestry, logging and related services (02)': 'p02',
 'Fish and other fishing products; services incidental of fishing (05)': 'p05',
 'Anthracite': 'p10.a',
 'Coking Coal': 'p10.b',
 'Other Bituminous Coal': 'p10.c',
 'Sub-Bituminous Coal': 'p10.d',
 'Patent Fuel': 'p10.e',
 'Lignite/Brown Coal': 'p10.f',
 'BKB/Peat Briquettes': 'p10.g',
 'Peat': 'p10.h',
 'Crude petroleum and services related to crude oil extraction, excluding surveying': 'p11.a',
 'Natural gas and services related to n

In [24]:
co2_final_total_products_flipped['exio code'] = co2_final_total_products_flipped['sector'].map(codes_dict)
co2_final_total_products_flipped.columns = ['country', 'product_name', 'total_co2', 'exio_code']
carbon_df = co2_final_total_products_flipped
carbon_df

,country,product_name,total_co2,exio_code
0,AT,Paddy rice,7.181080e+04,p01.a
1,AT,Wheat,2.359538e+06,p01.b
2,AT,Cereal grains nec,4.216438e+06,p01.c
3,AT,"Vegetables, fruit, nuts",1.615690e+07,p01.d
4,AT,Oil seeds,8.588490e+06,p01.e
...,...,...,...,...
7982,WM,Transportation services via pipelines,2.706219e+08,p60.3
7983,WM,Sea and coastal water transportation services,3.811673e+08,p61.1
7984,WM,Inland water transportation services,2.510483e+08,p61.2
7985,WM,Air transport services (62),4.015510e+07,p62


# Map and replace country codes

solving discrepancy between 3-letter and 2-letter


In [25]:
df_country = pd.read_csv('../Data/3_Trade/country_codes_V202102.csv', encoding='latin-1')
df_country

,country_code,country_name_abbreviation,country_name_full,iso_2digit_alpha,iso_3digit_alpha
0,4,Afghanistan,Afghanistan,AF,AFG
1,8,Albania,Albania,AL,ALB
2,12,Algeria,Algeria,DZ,DZA
3,16,American Samoa,American Samoa,AS,ASM
4,20,Andorra,Andorra,AD,AND
...,...,...,...,...,...
233,876,Wallis and Futuna Isds,Wallis and Futuna Islands,WF,WLF
234,882,Samoa,Samoa,WS,WSM
235,887,Yemen,Yemen,YE,YEM
236,891,Serbia and Montenegro,Serbia and Montenegro,CS,SCG


In [26]:
country_dict = pd.Series(df_country.iso_3digit_alpha.values,index=df_country.iso_2digit_alpha).to_dict()
country_dict

{'AF': 'AFG',
 'AL': 'ALB',
 'DZ': 'DZA',
 'AS': 'ASM',
 'AD': 'AND',
 'AO': 'AGO',
 'AG': 'ATG',
 'AZ': 'AZE',
 'AR': 'ARG',
 'AU': 'AUS',
 'AT': 'AUT',
 'BS': 'BHS',
 'BH': 'BHR',
 'BD': 'BGD',
 'AM': 'ARM',
 'BB': 'BRB',
 'BE': 'BEL',
 'BM': 'BMU',
 'BT': 'BTN',
 'BO': 'BOL',
 'BA': 'BIH',
 'BW': 'BWA',
 'BR': 'BRA',
 'BZ': 'BLZ',
 'IO': 'IOT',
 'SB': 'SLB',
 'VG': 'VGB',
 'BN': 'BRN',
 'BG': 'BGR',
 'MM': 'MMR',
 'BI': 'BDI',
 'BY': 'BLR',
 'KH': 'KHM',
 'CM': 'CMR',
 'CA': 'CAN',
 'CV': 'CPV',
 'KY': 'CYM',
 'CF': 'CAF',
 'LK': 'LKA',
 'TD': 'TCD',
 'CL': 'CHL',
 'CN': 'CHN',
 'CX': 'CXR',
 'CC': 'CCK',
 'CO': 'COL',
 'KM': 'COM',
 'YT': 'MYT',
 'CG': 'COG',
 'CD': 'COD',
 'CK': 'COK',
 'CR': 'CRI',
 'HR': 'HRV',
 'CU': 'CUB',
 'CY': 'CYP',
 'CS': 'SCG',
 'CZ': 'CZE',
 'BJ': 'BEN',
 'DK': 'DNK',
 'DM': 'DMA',
 'DO': 'DOM',
 'EC': 'ECU',
 'SV': 'SLV',
 'GQ': 'GNQ',
 'ET': 'ETH',
 'EE': 'EST',
 'FK': 'FLK',
 'FJ': 'FJI',
 'FI': 'FIN',
 'FR': 'FRA',
 'PF': 'PYF',
 'FQ': 'ATF',
 'DJ':

In [27]:
carbon_df['country'] = carbon_df['country'].map(country_dict)
carbon_df['year'] = 2010
carbon_df

,country,product_name,total_co2,exio_code,year
0,AUT,Paddy rice,7.181080e+04,p01.a,2010
1,AUT,Wheat,2.359538e+06,p01.b,2010
2,AUT,Cereal grains nec,4.216438e+06,p01.c,2010
3,AUT,"Vegetables, fruit, nuts",1.615690e+07,p01.d,2010
4,AUT,Oil seeds,8.588490e+06,p01.e,2010
...,...,...,...,...,...
7982,NaN,Transportation services via pipelines,2.706219e+08,p60.3,2010
7983,NaN,Sea and coastal water transportation services,3.811673e+08,p61.1,2010
7984,NaN,Inland water transportation services,2.510483e+08,p61.2,2010
7985,NaN,Air transport services (62),4.015510e+07,p62,2010


In [2]:
# number of failed country codes....

carbon_df['country'].isna().sum()

NameError: name 'carbon_df' is not defined

In [30]:
carbon_df.to_csv('../Data/2_Carbon/product_carbon_emissions_2010.csv', index=False)

# Code below can be IGNORED; was used for manually mapping products that had slightly different product names
Was used in OrganizeTrade

In [36]:
exio_df = carbon_df.loc[(carbon_df['exio_code'].isnull())]
exio_df = exio_df.head(20)
exio_df['product_name']

In [37]:
exio_df

,country,product_name,total_co2,exio_code,year
15,AUT,"Manure treatment (conventional), storage and l...",0.000000e+00,NaN,2010
16,AUT,"Manure treatment (biogas), storage and land ap...",0.000000e+00,NaN,2010
50,AUT,Re-processing of secondary wood material into ...,2.437059e+07,NaN,2010
52,AUT,Re-processing of secondary paper into new pulp,-4.849701e+05,NaN,2010
59,AUT,Re-processing of secondary plastic into new pl...,-4.957577e+06,NaN,2010
65,AUT,Re-processing of secondary glass into new glass,4.896631e+05,NaN,2010
69,AUT,Re-processing of ash into clinker,1.781885e+06,NaN,2010
72,AUT,Re-processing of secondary steel into new steel,1.787607e+06,NaN,2010
74,AUT,Re-processing of secondary preciuos metals int...,-3.961503e+02,NaN,2010
76,AUT,Re-processing of secondary aluminium into new ...,1.583428e+06,NaN,2010


In [38]:
exio_df.to_csv('../Data/2_Carbon/2010_products_no_code.csv', index=False)